In [18]:
import sys
sys.path.append(r"D:\MINJI\NETWORK RELIABILITY\BNS-JT-python")

import networkx as nx
import matplotlib.pyplot as plt
from BNS_JT import cpm, variable, operation
from networkx.algorithms.flow import shortest_augmenting_path
from BNS_JT import brc
import numpy as np

In [19]:
# Node and edge data
nodes = {
    "n1": (0, 0),
    "n2": (10, 10),
    "n3": (10, -10),
    "n4": (20, 0)
}

edges = {  
    "e1": ("n1", "n2"),   
    "e2": ("n2", "n1"),   
    "e3": ("n1", "n3"),   
    "e4": ("n3", "n1"),   
    "e5": ("n2", "n3"),   
    "e6": ("n3", "n2"),   
    "e7": ("n2", "n4"),   
    "e8": ("n4", "n2"),   
    "e9": ("n3", "n4"),   
    "e10": ("n4", "n3")   
}

probs = {
    'e1': {0: 0.01, 1: 0.99}, 
    'e2': {0: 0.01, 1: 0.99}, 
    'e3': {0: 0.01, 1: 0.99},
    'e4': {0: 0.01, 1: 0.99},
    'e5': {0: 0.05, 1: 0.95},
    'e6': {0: 0.05, 1: 0.95}, 
    'e7': {0: 0.05, 1: 0.95}, 
    'e8': {0: 0.05, 1: 0.95},
    'e9': {0: 0.10, 1: 0.90}, 
    'e10': {0: 0.10, 1: 0.90}
    }



# Assign fixed capacities
fixed_capacities = {
    "e1": 15, "e2": 15, 
    "e3": 15, "e4": 15,
    "e5": 7.5, "e6": 7.5,
    "e7": 15, "e8": 15, 
    "e9": 15, "e10": 15
}

# Edge Pairs for Same Capacity
edge_pairs = [
    ("e1", "e2"), ("e3", "e4"), ("e5", "e6"), ("e7", "e8"), ("e9", "e10")
]

# Apply failure probability and update capacities for edge pairs
def random_arc_capacity(probs, fixed_capacities):
    
    updated_capacities = {}

    for edge1, edge2 in edge_pairs:
        base_capacity = fixed_capacities[edge1]  
        fail_prob = probs[edge1][0]  

        if np.random.rand() < fail_prob:
            updated_capacities[edge1] = 0
            updated_capacities[edge2] = 0

        else:
            updated_capacities[edge1] = base_capacity
            updated_capacities[edge2] = base_capacity

    return updated_capacities

edge_capacities = random_arc_capacity(probs, fixed_capacities)
for edge, capacity in edge_capacities.items():
    print(f"Edge: {edge}, Final Capacity: {capacity}")



# Compute distances
def euclidean_distance(node1, node2):
    x1, y1 = nodes[node1]
    x2, y2 = nodes[node2]
    return round(((x2 - x1)**2 + (y2 - y1)**2)**0.5, 2)

arc_distance = {edge_name: euclidean_distance(u, v) for edge_name, (u, v) in edges.items()}   



# Create the graph
G = nx.DiGraph()

for node, position in nodes.items():
    G.add_node(node, pos=position)

for edge_name, (u, v) in edges.items(): 
      
    distance = arc_distance[edge_name]   
    capacity = edge_capacities[edge_name]  
    G.add_edge(u, v, weight=distance, capacity=capacity)   



# Calculate maximum allowable distance
# Demand data
demand = {
    'k1': {'origin': 'n1', 'destination': 'n4', 'amount': 28},
    'k2': {'origin': 'n2', 'destination': 'n3', 'amount': 18},
    'k3': {'origin': 'n1', 'destination': 'n3', 'amount': 11},
    'k4': {'origin': 'n3', 'destination': 'n4', 'amount': 16},
    'k5': {'origin': 'n4', 'destination': 'n1', 'amount': 20},
    'k6': {'origin': 'n3', 'destination': 'n2', 'amount': 14},
    'k7': {'origin': 'n3', 'destination': 'n1', 'amount': 11},
    'k8': {'origin': 'n4', 'destination': 'n3', 'amount': 18}
}

# Maximum allowable delay time 6 minutes / average velocity 149 km/h
avg_velo = 149
for commodity, info in demand.items():
    shortest_distance = nx.shortest_path_length(G, source=info['origin'], target=info['destination'], weight='weight')
    max_allowable_time = (shortest_distance * 60) / avg_velo + 6  # 6 minutes extra
    max_distance = max_allowable_time * avg_velo / 60



# Plot the network
plt.figure(figsize=(8, 6))
pos = nx.get_node_attributes(G, 'pos')
nx.draw_networkx_nodes(G, pos, node_size=450, node_color="lightblue")
nx.draw_networkx_edges(
    G, pos,
    edgelist=list(edges.values()),  
    arrowstyle='-|>',
    arrowsize=15,
    connectionstyle='arc3,rad=0.1',
    min_target_margin=10,
    min_source_margin=10
)
edge_labels = {(u, v): f"{arc_distance[edge_name]} km, Cap: {edge_capacities[edge_name]}" 
               for edge_name, (u, v) in edges.items()}
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8, label_pos=0.5)
nx.draw_networkx_labels(G, pos, font_size=9)
plt.title("Network with Bidirectional Arcs")
plt.show()

Edge: e1, Final Capacity: 15
Edge: e2, Final Capacity: 15
Edge: e3, Final Capacity: 15
Edge: e4, Final Capacity: 15
Edge: e5, Final Capacity: 7.5
Edge: e6, Final Capacity: 7.5
Edge: e7, Final Capacity: 15
Edge: e8, Final Capacity: 15
Edge: e9, Final Capacity: 15
Edge: e10, Final Capacity: 15


In [20]:
varis = {}
for k, v in edges.items():
    varis[k] = variable.Variable( name=k, values = [0, 1]) # values: edge flow capacity

In [21]:
def net_conn(comps_st, od_pair, edges, varis): # maximum flow analysis

    G = nx.Graph()
    for k,x in comps_st.items():
        G.add_edge(edges[k][0], edges[k][1]) # we add each edge
        G[edges[k][0]][edges[k][1]]['capacity'] = varis[k].values[x] # define capacity as 0 if state = 0 or 1 if state = 1

    # perform maximum flow analysis between the OD pair
    G.add_edge(od_pair[1], 'new_d', capacity=1) # add a new edge with capacity 1 to ensure we find only ONE path. 
    f_val, f_dict = nx.maximum_flow(G, od_pair[0], 'new_d', capacity='capacity', flow_func=shortest_augmenting_path)

    if f_val > 0: # if the flow between the OD pair is greater than 0, the two nodes are connected
        sys_st = 's'

        # We can infer an associated minimum survival rule in case of network connectivity.
        min_comps_st = {} 
        for k, x in comps_st.items():
            k_flow = max([f_dict[edges[k][0]][edges[k][1]], f_dict[edges[k][1]][edges[k][0]]])
            if k_flow > 0: # the edges with flows > 0 consitute a minimum survival rule.
                min_comps_st[k] = 1

    else:
        sys_st = 'f'

        # In case of system failure, obtaining a minimum failure rule is not straightforward.
        min_comps_st = None

    return f_val, sys_st, min_comps_st

In [ ]:
def MCNF_systemfunc(comps_st, edges, demand, max_distance, arc_distance):
    from gurobipy import Model, GRB, quicksum

    # Create Gurobi model
    model = Model("Network Flow Optimization")
    
    sampled_edge_capacities = random_arc_capacity(probs, fixed_capacities)
    arcs = [(u, v) for _, (u, v) in edges.items()]
    arc_capacity = {(u, v): sampled_edge_capacities[edge_name]
                    for edge_name, (u, v) in edges.items()}
    
    # Define variables
    flow = {}
    unmet_demand = {}
    for k, info in demand.items():
        unmet_demand[k] = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name=f"unsatisfied_{k}")
        for i, j in arcs:
            # Determine capacity based on component state
            capacity = arc_capacity.get((i, j), 0) * comps_st[arcs.index((i, j))]
            flow[k, i, j] = model.addVar(lb=0, ub=capacity, vtype=GRB.CONTINUOUS, name=f"flow_{k}_{i}_{j}")

    # Objective function: Minimize expected loss
    cost_coefficient = {
        k: info['amount'] * nx.shortest_path_length(G, source=info['origin'], target=info['destination'], weight='weight') * 0.1723
        for k, info in demand.items()
    }
    
    model.setObjective(
        quicksum(cost_coefficient[k] * unmet_demand[k] for k in demand),
        GRB.MINIMIZE
    )

    # Extract all nodes from edge values
    nodes = set(node for edge in edges.values() for node in edge)

    # Constraint 1: Flow conservation
    for k, info in demand.items():
        origin = info['origin']
        destination = info['destination']
        amount = info['amount']
        for node in nodes: 
            inflow = quicksum(flow[k, i, j] for i, j in arcs if j == node)
            outflow = quicksum(flow[k, i, j] for i, j in arcs if i == node)
            if node == origin:
                model.addConstr(outflow - inflow == amount - unmet_demand[k])
            elif node == destination:
                model.addConstr(outflow - inflow == - amount + unmet_demand[k])
            else:
                model.addConstr(outflow - inflow == 0)

    # Constraint 2: Arc capacity limits
    for i, j in arcs:
        model.addConstr(quicksum(flow[k, i, j] for k in demand) <= arc_capacity.get((i, j), 0))

    # Constraint 3: Distance limits
    for k, info in demand.items():
        origin = info['origin']
        distance_expr = quicksum(arc_distance.get((i, j), 0) * flow[k, i, j] for i, j in arcs)
        total_flow = quicksum(flow[k, i, j] for i, j in arcs if i == origin)
        model.addConstr(distance_expr <= max_distance * total_flow)

    # Perform optimization
    model.optimize()

    # Process results
    if model.status == GRB.OPTIMAL:
        expected_loss = model.objVal
        flows = {
            k: {arc: flow[k, arc[0], arc[1]].X for arc in arcs} for k in demand
        }

        # Define system state based on expected_loss
        if expected_loss <= 2400:
            sys_st = 'threshold 1'
        elif 2400 < expected_loss <= 2600:
            sys_st = 'threshold 2'
        elif 2600 < expected_loss <= 2800:
            sys_st = 'threshold 3'
        elif 2800 < expected_loss <= 3000:
            sys_st = 'threshold 4'
        else:
            sys_st = 'threshold 5'

        return expected_loss, sys_st, flows
    else:
        return None, None, None

In [23]:
# Solve MCNF optimization for the current sample
sys_fun = lambda comps_st : MCNF_systemfunc(
    comps_st=
    edges = edges,
    demand = demand,
    max_distance = max_distance
    arc_distance = arc_distance
)

In [24]:
brs, rules, sys_res, monitor = brc.run(varis, probs, sys_fun, max_sf=np.inf, max_nb=np.inf, pf_bnd_wr=0.0)

NameError: cannot access free variable 'nodes' where it is not associated with a value in enclosing scope